# ACO Hyperparameter Tuning for Graph Coloring Problem

This notebook performs hyperparameter tuning for Ant Colony Optimization (ACO) algorithm applied to the Graph Coloring Problem using Optuna.

## 1. Environment Setup

In [14]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

Running in Google Colab: False
Local environment detected. Using local paths.


In [15]:
# Configure paths for data, studies, results, and figures based on the execution environment.
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

Base Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3
Code Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code
Data Root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Path verification: OK


In [16]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0 scikit-learn==1.4.0 scipy==1.11.4
    print("Packages installed successfully!")

In [17]:
# Import Required Libraries
from pathlib import Path
import multiprocessing
from datetime import datetime
from IPython.display import Image, display, Markdown, HTML

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Study Configuration

Define dataset selection, study name, and hyperparameter search space.

In [18]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'limited_dataset'  # Change to 'main_dataset' for full tuning

# Study continuation mode
# Option 1: Continue existing study - set CONTINUE_STUDY to the study name
# Example: CONTINUE_STUDY = 'aco_study_tiny_dataset_20251129_172927'
# Option 2: Start new study - set CONTINUE_STUDY = None
CONTINUE_STUDY = None  # Set to study name to continue, or None for new study

# Study name (used for new studies or when CONTINUE_STUDY is None)
if CONTINUE_STUDY:
    STUDY_NAME = CONTINUE_STUDY
    print(f"Continuing existing study: {STUDY_NAME}")
else:
    STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    print(f"Creating new study: {STUDY_NAME}")

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 20

# Number of random exploration trials before optimization starts
N_STARTUP_TRIALS = 4

# ACO verbose setting 
ACO_VERBOSE = False   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    'iterations': {
        'type': 'int',
        'low': 50,
        'high': 200,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 6.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.01,
        'high': 0.4,
    },
    'ant_count': {
        'type': 'int',
        'low': 10,
        'high': 100,
    },
    'Q': {
        'type': 'float',
        'low': 1,
        'high': 10.0,
    },
    'patience': {
        'type': 'float',
        'low': 0.40,
        'high': 0.50,
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"  Random Exploration Trials: {N_STARTUP_TRIALS} (before optimization)")
print(f"\nHyperparameters to Optimize:")
for param_name, param_spec in PARAM_CONFIG.items():
    print(f"  {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

Creating new study: aco_study_limited_dataset_20251130_222022
Configuration:
  Dataset: limited_dataset
  Study Name: aco_study_limited_dataset_20251130_222022
  Optuna Trials: 20
  Random Exploration Trials: 4 (before optimization)

Hyperparameters to Optimize:
  iterations: [50, 200] (int)
  alpha: [0.5, 3.0] (float)
  beta: [1.0, 6.0] (float)
  rho: [0.01, 0.4] (float)
  ant_count: [10, 100] (int)
  Q: [1, 10.0] (float)
  patience: [0.4, 0.5] (float)


In [19]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    direction='minimize',  # We want to minimize the number of colors used
    n_startup_trials=N_STARTUP_TRIALS  # Number of random trials before optimization
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")

# Load tuning dataset once (before optimization)
print("\nLoading tuning dataset...")
data_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)
tuning_graphs, tabu_best_values = data_loader.load_tuning_dataset()
print(f"Loaded {len(tuning_graphs)} graphs for tuning\n")

# Wrapper function to pass tuning graphs and tabu best values to objective function
def objective_wrapper(trial, params, **kwargs):
    return aco_objective_function(
        trial=trial,
        params=params,
        tuning_graphs=tuning_graphs,
        aco_class=ACOGraphColoring,
        verbose=ACO_VERBOSE,
        tabu_best_values=tabu_best_values,
        n_startup_trials=N_STARTUP_TRIALS
    )

print("Objective function wrapper ready!")

Tuner initialized with study: aco_study_limited_dataset_20251130_222022
Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Loading tuning dataset...

Loading Tuning Dataset: limited_dataset
  gc_500_9:
    Nodes: 500, Edges: 112224
    Density: 0.8996, Connected: True

Graphs ordered by size (smallest to largest):
  1. gc_500_9: 500 nodes, 112224 edges


Loaded Tabu Tuning Best Results: 8 graphs
  gc_1000_7: 195 colors (from tabu search tuning)
  gc_100_9: 38 colors (from tabu search tuning)
  gc_20_9: 10 colors (from tabu search tuning)
  gc_250_9: 85 colors (from tabu search tuning)
  gc_4_1: 2 colors (from tabu search tuning)
  gc_500_9: 162 colors (from tabu search tuning)
  gc_50_9: 21 colors (from tabu search tuning)
  gc_70_9: 28 colors (from tabu search tuning)

Loaded 1 graphs for tuning

Objective function wrapper ready!


## 3. Run Hyperparmeters Tuning

In [20]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS,
    n_jobs=6
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

[I 2025-11-30 22:20:22,921] A new study created in Journal with name: aco_study_limited_dataset_20251130_222022



Starting hyperparameter optimization with 20 trials...
✓ Created new study 'aco_study_limited_dataset_20251130_222022'
  Storage: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_222022/aco_study_limited_dataset_20251130_222022.log
  Random exploration trials: 4 (before optimization)
Trial Summary:
  Successful: 0/20
  Total executed: 0
  Remaining needed: 20

Starting optimization with 20 trials...


  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0: Random Exploration (Startup 1/4)
Parameters:
  iterations: 182
  alpha: 2.92695065535295
  beta: 5.3875439950493735
  rho: 0.016260788497549396
  ant_count: 36
  Q: 3.644806503887892
  patience: 0.40569264263409405
----------------------------------------------------------------------

💡 Trial 0 | Graph 'gc_500_9' | [1/1] | Starting (nodes=500, edges=112224, tabu_best=162)

Trial 1: Random Exploration (Startup 2/4)
Parameters:
  iterations: 190
  alpha: 2.8214851285557505
  beta: 2.1503725770932762
  rho: 0.19750400517999475
  ant_count: 84
  Q: 8.809330330937161
  patience: 0.4973996622700807
----------------------------------------------------------------------

💡 Trial 1 | Graph 'gc_500_9' | [1/1] | Starting (nodes=500, edges=112224, tabu_best=162)

Trial 4: TPE Optimization
Parameters:
  iterations: 179
  alpha: 1.7871164190800775
  beta: 1.210299088277307
  rho: 0.15882743626116144
  ant_count: 90
  Q: 2.2959575418915783
  patience: 0.4521369833954371
-------------------

  0%|          | 0/20 [00:36<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Generate all optimization plots
print("\nGenerating optimization visualization plots...")
print("=" * 70)
tuner.generate_plots(recreate=True)
print("=" * 70)


Generating optimization visualization plots...

Generating visualization plots...
Optimization history saved to: history.png
Parameter importances saved to: importances.png
Slice plot saved to: slice.png
Timeline plot saved to: timeline.png
All plots generated successfully.



### Best Trial Results

In [ ]:
# Display best trial graphs and metrics


# Get best trial number
best_trial_number = tuner.study.best_trial.number
best_trial_dir = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'results' / f'trial_{best_trial_number:04d}'

print(f"Best Trial: {best_trial_number}")
print(f"Best Objective Value: {tuner.study.best_value}")
print("=" * 70)

# Display metrics for best trial
display(Markdown(f"#### Best Trial Metrics"))

metric_files = [
    ('color_count.png', 'Color Count per Graph'),
    ('execution_time.png', 'Execution Time per Graph'),
    ('conflicts.png', 'Conflicts per Graph')
]

for filename, title in metric_files:
    metric_path = best_trial_dir / filename
    if metric_path.exists():
        display(Markdown(f"**{title}**"))
        display(HTML(f'<img src="{metric_path}" style="width: 75%; display: block; margin: 0 auto;">'))

# Display colored graphs for best trial
display(Markdown(f"#### Best Trial: Colored Graph Solutions"))

# Find all graph files in best trial directory
graph_files = sorted(best_trial_dir.glob('graph_*.png'))
for graph_file in graph_files:
    graph_name = graph_file.stem.replace('graph_', '')
    display(Markdown(f"**{graph_name}**"))
    display(HTML(f'<img src="{graph_file}" style="width: 75%; display: block; margin: 0 auto;">'))

Best Trial: 6
Best Objective Value: 205.0


#### Best Trial Metrics

**Color Count per Graph**

**Execution Time per Graph**

**Conflicts per Graph**

#### Best Trial: Colored Graph Solutions

**gc_500_9**

### Display Study Visualizations

In [ ]:
# Display all study figures

study_figures_path = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'figures'

# List of all possible study figures with descriptions
figure_files = [
    ('history.png', 'Optimization History', 'Shows objective value progression across trials'),
    ('importances.png', 'Parameter Importances', 'Shows which hyperparameters impact results most'),
    ('slice.png', 'Slice Plots (All Parameters)', 'Shows how each parameter affects objective value'),
    ('timeline.png', 'Trial Timeline', 'Shows when each trial ran and how long it took')
]

print("Study Visualization Figures:")
print("=" * 70)

for filename, title, description in figure_files:
    figure_path = study_figures_path / filename
    if figure_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f"*{description}*"))
        display(HTML(f'<img src="{figure_path}" style="width: 75%; display: block; margin: 0 auto;">'))
        print(f"✓ {filename}")
    else:
        print(f"✗ {filename} (not generated - may require additional dependencies)")

print("=" * 70)

Study Visualization Figures:


### Optimization History

*Shows objective value progression across trials*

✓ history.png


### Parameter Importances

...

*Shows how each parameter affects objective value*

✓ slice.png


### Trial Timeline

*Shows when each trial ran and how long it took*

✓ timeline.png


## 4. Comprehensive Algorithms Comparison

Compare three algorithms on the testing dataset:
- **Greedy Algorithm**: Fast constructive heuristic
- **Tabu Search**: Single-solution metaheuristic with memory
- **ACO**: Population-based metaheuristic (using best parameters from tuning)

Each algorithm runs 5 times on all test graphs to calculate statistics and generate comparison plots.

In [ ]:
# Import the comprehensive testing module
from comprehensive_testing import run_comprehensive_testing

# Set up testing directories - save comparison inside study path
comparison_output_dir = DATA_ROOT / 'studies' / STUDY_NAME / 'algorithm_comparison'

print(f"Data root: {DATA_ROOT}")
print(f"Dataset: {DATASET_NAME}")
print(f"Comparison output directory: {comparison_output_dir}")
print(f"  (Results will be saved in study: {STUDY_NAME})")

Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data
Dataset: limited_dataset
Comparison output directory: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison
  (Results will be saved in study: aco_study_limited_dataset_20251130_212043)


In [ ]:
# Prepare ACO parameters from best trial
aco_test_params = {
    'iterations': int(best_params['iterations']),
    'alpha': best_params['alpha'],
    'beta': best_params['beta'],
    'rho': best_params['rho'],
    'ant_count': int(best_params['ant_count']),
    'Q': best_params['Q'],
    'patience': best_params['patience']
}

print("\nACO Parameters for Testing (from best trial):")
for param, value in aco_test_params.items():
    print(f"  {param}: {value}")


ACO Parameters for Testing (from best trial):
  iterations: 89
  alpha: 0.6046246627750127
  beta: 5.906264522607428
  rho: 0.3124478342288473
  ant_count: 55
  Q: 8.521663286568902
  patience: 0.4003136761769846


In [24]:
# Run comprehensive testing: 5 repetitions per algorithm per graph
# Results are cached in JSON - set force_rerun=True to re-execute tests
NUM_REPETITIONS = 1
FORCE_RERUN = False  # Set to True to ignore cached results and rerun testing

print(f"\n{'='*80}")
print(f"Running comprehensive testing with {NUM_REPETITIONS} repetitions per algorithm")
if FORCE_RERUN:
    print("⚠ FORCE_RERUN=True: Will ignore cached results and rerun all tests")
else:
    print("ℹ Using cached results if available (set FORCE_RERUN=True to rerun)")
print(f"{'='*80}\n")

df_comparison_results, df_comparison_stats = run_comprehensive_testing(
    data_root=str(DATA_ROOT),
    dataset_name=DATASET_NAME,
    output_dir=str(comparison_output_dir),
    num_repetitions=NUM_REPETITIONS,
    aco_params=aco_test_params,
    force_rerun=FORCE_RERUN
)

print("\n✓ Comprehensive testing complete!")


Running comprehensive testing with 1 repetitions per algorithm
ℹ Using cached results if available (set FORCE_RERUN=True to rerun)


Loaded Best Known Results: 3 graphs
  dsjc1000.5: 85 colors
  dsjc250.5: 28 colors
  dsjc500.9: 126 colors


Loading Testing Dataset: limited_dataset
  dsjc250.5:
    Nodes: 250, Edges: 15668
    Density: 0.5034, Connected: True

Graphs ordered by size (smallest to largest):
  1. dsjc250.5: 250 nodes, 15668 edges

Checking for existing test results...
COMPREHENSIVE ALGORITHM TESTING
Found 1 test graphs
Running 1 repetitions per graph per algorithm

Processing: dsjc250.5
  Nodes: 250, Edges: 15668
  Running Greedy...
    Rep 1: colors=42, conflicts=0, time=0.015s
  Running Tabu Search...
    Rep 1: colors=35, conflicts=0, time=246.173s
  Running ACO...
💬 Trial 0 | Graph 'gc_500_9' | [1/1] | Iteration 50/182: iter_best=216, global_best=210 - ⚠️ vs Tabu: 162 (+48)
💬 Trial 5 | Graph 'gc_500_9' | [1/1] | Iteration 50/179: iter_best=233, global_best=229 - ⚠️ v

[W 2025-11-30 22:52:04,749] Trial 5 failed with parameters: {'iterations': 179, 'alpha': 1.6390853449202585, 'beta': 4.037391928015461, 'rho': 0.10671221878255192, 'ant_count': 66, 'Q': 8.62023194622315, 'patience': 0.43873891975713913} because of the following error: ZMQError('Socket operation on non-socket').
Traceback (most recent call last):
  File "/Users/mahdy/projects/meta_graph_coloring_antcol/venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py", line 237, in wrapped_objective
    return objective_func(
           ^^^^^^^^^^^^^^^
  File "/var/folders/xh/_mzdfyfn0x51nq60wcf392zm0000gn/T/ipykernel_30550/3821460395.py", line 20, in objective_wrapper
    return aco_objective_function(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/o

💬 Trial 5 | Graph 'gc_500_9' | [1/1] | Iteration 60/179: iter_best=237, global_best=229 - ⚠️ vs Tabu: 162 (+67)💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 60/179: iter_best=249, global_best=249 - ⚠️ vs Tabu: 162 (+87)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 60/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 2 | Graph 'gc_500_9' | [1/1] | Iteration 60/153: iter_best=228, global_best=225 - ⚠️ vs Tabu: 162 (+63)
💬 Iteration 30/89: iter_best=66, global_best=57
💬 Iteration 40/89: iter_best=67, global_best=57
💬 Trial 0 | Graph 'gc_500_9' | [1/1] | Iteration 70/182: iter_best=215, global_best=210 - ⚠️ vs Tabu: 162 (+48)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 70/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 70/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 2 | Graph 'gc_500_9' | [1/1] | Iteration 70/153: iter_best=229, global_best=225 - ⚠️ vs Tabu: 162 (+63)
💬 Iterati

### 6.1 Display Statistics Summary

In [25]:
# Display statistics table
from IPython.display import display, Markdown

print("Statistics Summary:")
display(df_comparison_stats)

# Print formatted statistics (without markdown)
print("\n" + "="*80)
print(df_comparison_stats.to_string(index=False))
print("="*80)

Statistics Summary:


,graph,best_known,algorithm,best_colors,avg_colors,std_colors,avg_conflicts,avg_time,std_time,deviation_best,deviation_avg
0,dsjc250.5,28,Greedy,42,42.0,NaN,0.0,0.015463,NaN,50.000000,50.000000
1,dsjc250.5,28,Tabu Search,35,35.0,NaN,0.0,246.173195,NaN,25.000000,25.000000
2,dsjc250.5,28,ACO,57,57.0,NaN,0.0,317.718126,NaN,103.571429,103.571429



    graph  best_known   algorithm  best_colors  avg_colors  std_colors  avg_conflicts   avg_time  std_time  deviation_best  deviation_avg
dsjc250.5          28      Greedy           42        42.0         NaN            0.0   0.015463       NaN       50.000000      50.000000
dsjc250.5          28 Tabu Search           35        35.0         NaN            0.0 246.173195       NaN       25.000000      25.000000
dsjc250.5          28         ACO           57        57.0         NaN            0.0 317.718126       NaN      103.571429     103.571429


### 6.2 Display Comparison Visualizations

In [26]:
# Display algorithm comparison plots (4 comparison metrics)
from IPython.display import Image

# Define all comparison plots
comparison_plots = [
    ('comparison_best_colors.png', 'Best Color Count Comparison (with Best Known Solutions)'),
    ('comparison_avg_colors.png', 'Average Color Count Comparison (with Std Dev)'),
    ('comparison_execution_time.png', 'Execution Time Comparison (log scale)'),
    ('comparison_conflicts.png', 'Conflict Count Comparison')
]

print("Algorithm Comparison Metrics:")
print("="*80)

for filename, title in comparison_plots:
    plot_path = comparison_output_dir / filename
    if plot_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f'<img src="{plot_path}" style="width: 75%; display: block; margin: 0 auto;"/>'))
    else:
        print(f"⚠ {title} not found: {filename}")

print("="*80)

Algorithm Comparison Metrics:


### Best Color Count Comparison (with Best Known Solutions)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison/comparison_best_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Average Color Count Comparison (with Std Dev)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison/comparison_avg_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Execution Time Comparison (log scale)

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison/comparison_execution_time.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Conflict Count Comparison

<img src="/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison/comparison_conflicts.png" style="width: 75%; display: block; margin: 0 auto;"/>

### 6.3 Results Summary and File Locations

In [27]:
# Print comprehensive testing results summary
print("="*80)
print("COMPREHENSIVE TESTING RESULTS SUMMARY")
print("="*80)

print(f"\n📁 Output Directory: {comparison_output_dir}")
print(f"   (Saved in study: {STUDY_NAME})")

print(f"\n📊 Generated Files:")
print(f"   - comparison_results.json (complete results - cached for reuse)")
print(f"   - comprehensive_test_results.csv (all {len(df_comparison_results)} individual runs)")
print(f"   - statistics_summary.csv (aggregated statistics)")
print(f"   - comparison_best_colors.png (best color count comparison)")
print(f"   - comparison_avg_colors.png (average color count with std dev)")
print(f"   - comparison_execution_time.png (execution time comparison)")
print(f"   - comparison_conflicts.png (conflict count comparison)")
print(f"   - color_distribution_boxplots.png (distribution analysis)")

print(f"\n💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.")
print(f"   Set FORCE_RERUN=True to ignore cache and rerun all tests.")

print(f"\n🎯 Summary by Algorithm:")
for algo_name in df_comparison_stats['algorithm'].unique():
    df_algo = df_comparison_stats[df_comparison_stats['algorithm'] == algo_name]
    best_colors = df_algo['best_colors'].min()
    avg_colors = df_algo['avg_colors'].mean()
    avg_time = df_algo['avg_time'].mean()
    print(f"   {algo_name}:")
    print(f"      Best colors: {best_colors}")
    print(f"      Avg colors: {avg_colors:.2f}")
    print(f"      Avg time: {avg_time:.4f}s")

print("\n" + "="*80)

COMPREHENSIVE TESTING RESULTS SUMMARY

📁 Output Directory: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_212043/algorithm_comparison
   (Saved in study: aco_study_limited_dataset_20251130_222022)

📊 Generated Files:
   - comparison_results.json (complete results - cached for reuse)
   - comprehensive_test_results.csv (all 3 individual runs)
   - statistics_summary.csv (aggregated statistics)
   - comparison_best_colors.png (best color count comparison)
   - comparison_avg_colors.png (average color count with std dev)
   - comparison_execution_time.png (execution time comparison)
   - comparison_conflicts.png (conflict count comparison)
   - color_distribution_boxplots.png (distribution analysis)

💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.
   Set FORCE_RERUN=True to ignore cache and rerun all tests.

🎯 Summary by Algorithm:
   Greedy:
      Best colors: 42
      Avg colors: 42.00
      Avg 

💬 Trial 0 | Graph 'gc_500_9' | [1/1] | Iteration 80/182: iter_best=216, global_best=210 - ⚠️ vs Tabu: 162 (+48)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 80/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 80/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 2 | Graph 'gc_500_9' | [1/1] | Iteration 80/153: iter_best=229, global_best=225 - ⚠️ vs Tabu: 162 (+63)
💬 Trial 0 | Graph 'gc_500_9' | [1/1] | Iteration 90/182: iter_best=212, global_best=210 - ⚠️ vs Tabu: 162 (+48)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 90/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 90/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 2 | Graph 'gc_500_9' | [1/1] | Iteration 90/153: iter_best=231, global_best=225 - ⚠️ vs Tabu: 162 (+63)


[I 2025-11-30 22:57:54,342] Trial 2 finished with value: 225.0 and parameters: {'iterations': 153, 'alpha': 1.145161345562828, 'beta': 4.25854242707404, 'rho': 0.06426959367329471, 'ant_count': 89, 'Q': 1.8824201202676503, 'patience': 0.48392176718402535}. Best is trial 2 with value: 225.0.



❌ Trial 2 | Graph 'gc_500_9' | [1/1] |  Early stopping: No improvement for 74 iterations
🔹 Trial 2 | Graph 'gc_500_9' | [1/1] | ✓ Result: 225 colors in 94 iterations - ⚠️ Less than Tabu Best: 162 (+63) | ⏱ 2251.00s

----------------------------------------------------------------------
Trial 2 Summary:
  Total colors across all graphs: 225
  Average colors per graph: 225.00

    Saved trial visualizations: 3 metric plots, 1 graph plots
💬 Trial 0 | Graph 'gc_500_9' | [1/1] | Iteration 100/182: iter_best=217, global_best=210 - ⚠️ vs Tabu: 162 (+48)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 100/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 100/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)


[I 2025-11-30 22:58:51,385] Trial 0 finished with value: 210.0 and parameters: {'iterations': 182, 'alpha': 2.92695065535295, 'beta': 5.3875439950493735, 'rho': 0.016260788497549396, 'ant_count': 36, 'Q': 3.644806503887892, 'patience': 0.40569264263409405}. Best is trial 0 with value: 210.0.



❌ Trial 0 | Graph 'gc_500_9' | [1/1] |  Early stopping: No improvement for 73 iterations
🔹 Trial 0 | Graph 'gc_500_9' | [1/1] | ✓ Result: 210 colors in 103 iterations - ⚠️ Less than Tabu Best: 162 (+48) | ⏱ 2308.44s

----------------------------------------------------------------------
Trial 0 Summary:
  Total colors across all graphs: 210
  Average colors per graph: 210.00

    Saved trial visualizations: 3 metric plots, 1 graph plots
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 110/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 110/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
💬 Trial 1 | Graph 'gc_500_9' | [1/1] | Iteration 120/190: iter_best=241, global_best=241 - ⚠️ vs Tabu: 162 (+79)
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 120/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)

❌ Trial 1 | Graph 'gc_500_9' | [1/1] |  Early stopping: No improvement for 94 iterations
🔹 Trial 1 | Grap

[I 2025-11-30 23:01:09,315] Trial 1 finished with value: 241.0 and parameters: {'iterations': 190, 'alpha': 2.8214851285557505, 'beta': 2.1503725770932762, 'rho': 0.19750400517999475, 'ant_count': 84, 'Q': 8.809330330937161, 'patience': 0.4973996622700807}. Best is trial 0 with value: 210.0.


💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 130/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)
    Saved trial visualizations: 3 metric plots, 1 graph plots
💬 Trial 4 | Graph 'gc_500_9' | [1/1] | Iteration 140/179: iter_best=245, global_best=245 - ⚠️ vs Tabu: 162 (+83)


[I 2025-11-30 23:02:13,792] Trial 4 finished with value: 245.0 and parameters: {'iterations': 179, 'alpha': 1.7871164190800775, 'beta': 1.210299088277307, 'rho': 0.15882743626116144, 'ant_count': 90, 'Q': 2.2959575418915783, 'patience': 0.4521369833954371}. Best is trial 0 with value: 210.0.



❌ Trial 4 | Graph 'gc_500_9' | [1/1] |  Early stopping: No improvement for 80 iterations
🔹 Trial 4 | Graph 'gc_500_9' | [1/1] | ✓ Result: 245 colors in 147 iterations - ⚠️ Less than Tabu Best: 162 (+83) | ⏱ 2510.50s

----------------------------------------------------------------------
Trial 4 Summary:
  Total colors across all graphs: 245
  Average colors per graph: 245.00

    Saved trial visualizations: 3 metric plots, 1 graph plots
